In [1]:
%load_ext autoreload
%autoreload 2

# For Stack

Generate mock responses

In [2]:
from s1_frame_enumerator import S1Frame, query_slc_metadata_over_frame

In [3]:
from s1_frame_enumerator import S1Frame, query_slc_metadata_over_frame
import json

In [4]:
def generate_mock_data(frame_id):
    frame = S1Frame(frame_id)
    result = query_slc_metadata_over_frame(frame)
    out_path = f'frame_{frame_id}_asf_results.json'
    json.dump(result, open(out_path, 'w'))
    return out_path

In [5]:
#generate_mock_data(21249)
#generate_mock_data(22439)
generate_mock_data(13403)
generate_mock_data(13404)
generate_mock_data(9847)
generate_mock_data(9848)

'frame_9848_asf_results.json'

# For Enumeration

Generate Small Stack

In [6]:
from shapely.geometry import Point
import geopandas as gpd
from rasterio.crs import CRS
from s1_frame_enumerator import get_overlapping_s1_frames, get_s1_stack

aoi_geo = Point(-120, 35).buffer(.5)
df_aoi = gpd.GeoDataFrame(geometry=[aoi_geo], crs=CRS.from_epsg(4326))

In [7]:
frames = get_overlapping_s1_frames(aoi_geo, track_numbers=[137])
len(frames)

2

In [20]:
for frame in frames:
    print(frame.frame_id)

21248
21249


In [8]:
df_stack = get_s1_stack(frames)
df_stack.head()

,slc_id,start_time,stop_time,url,track_number,polarization,orbit,beam_mode,size_gb,stack_repeat_pass_id,repeat_pass_date,geometry
0,S1A_IW_SLC__1SDV_20150401T015800_20150401T0158...,2015-04-01 01:58:00+00:00,2015-04-01 01:58:30+00:00,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...,137,VV+VH,5284,IW,4.907014,3,2015-04-01,"POLYGON ((-121.18324 35.18838, -120.76283 33.3..."
1,S1A_IW_SLC__1SDV_20150401T015828_20150401T0158...,2015-04-01 01:58:28+00:00,2015-04-01 01:58:55+00:00,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...,137,VV+VH,5284,IW,4.697075,4,2015-04-01,"POLYGON ((-121.54370 36.73265, -121.14296 35.0..."
2,S1A_IW_SLC__1SSV_20150507T015751_20150507T0158...,2015-05-07 01:57:51+00:00,2015-05-07 01:58:21+00:00,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...,137,VV,5809,IW,2.652423,5,2015-05-07,"POLYGON ((-121.02775 34.52478, -120.61049 32.7..."
3,S1A_IW_SLC__1SSV_20150507T015818_20150507T0158...,2015-05-07 01:58:18+00:00,2015-05-07 01:58:45+00:00,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...,137,VV,5809,IW,2.437214,5,2015-05-07,"POLYGON ((-121.37492 36.01932, -120.98943 34.3..."
4,S1A_IW_SLC__1SSV_20150507T015843_20150507T0159...,2015-05-07 01:58:43+00:00,2015-05-07 01:59:11+00:00,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...,137,VV,5809,IW,2.568650,5,2015-05-07,"POLYGON ((-121.74913 37.56177, -121.34393 35.8..."


In [9]:
df_stack.repeat_pass_date.unique()

array([datetime.date(2015, 4, 1), datetime.date(2015, 5, 7),
       datetime.date(2015, 5, 31), datetime.date(2015, 6, 12),
       datetime.date(2015, 6, 24), datetime.date(2015, 7, 18),
       datetime.date(2015, 7, 30), datetime.date(2015, 8, 11),
       datetime.date(2015, 8, 23), datetime.date(2015, 9, 4),
       datetime.date(2015, 9, 16), datetime.date(2015, 9, 28),
       datetime.date(2015, 10, 10), datetime.date(2015, 11, 3),
       datetime.date(2015, 11, 15), datetime.date(2015, 11, 27),
       datetime.date(2015, 12, 21), datetime.date(2016, 1, 14),
       datetime.date(2016, 1, 26), datetime.date(2016, 2, 7),
       datetime.date(2016, 2, 19), datetime.date(2016, 3, 2),
       datetime.date(2016, 3, 14), datetime.date(2016, 3, 26),
       datetime.date(2016, 4, 7), datetime.date(2016, 4, 19),
       datetime.date(2016, 5, 1), datetime.date(2016, 5, 13),
       datetime.date(2016, 5, 25), datetime.date(2016, 6, 6),
       datetime.date(2016, 6, 30), datetime.date(2016, 7, 2

In [15]:
import datetime
cond_0 = df_stack.repeat_pass_date > datetime.date(2022, 9, 1)
cond_1 = df_stack.repeat_pass_date < datetime.date(2023, 1, 1)

df_stack_subset = df_stack[cond_0 & cond_1].reset_index(drop=True)
df_stack_subset.shape

(30, 12)

In [18]:
import pandas as pd
df_stack_subset.repeat_pass_date = pd.to_datetime(df_stack_subset.repeat_pass_date)
df_stack_subset.to_file('sample_stack_137.geojson', driver='GeoJSON')

In [19]:
df_stack_subset.repeat_pass_date

0    2022-09-03
1    2022-09-03
2    2022-09-03
3    2022-09-15
4    2022-09-15
5    2022-09-15
6    2022-09-27
7    2022-09-27
8    2022-09-27
9    2022-10-09
10   2022-10-09
11   2022-10-09
12   2022-10-21
13   2022-10-21
14   2022-10-21
15   2022-11-02
16   2022-11-02
17   2022-11-02
18   2022-11-14
19   2022-11-14
20   2022-11-14
21   2022-11-26
22   2022-11-26
23   2022-11-26
24   2022-12-08
25   2022-12-08
26   2022-12-08
27   2022-12-20
28   2022-12-20
29   2022-12-20
Name: repeat_pass_date, dtype: datetime64[ns]

In [33]:
from s1_frame_enumerator.ifg_enum import select_ifg_pair_from_stack
frames = [S1Frame(21248), S1Frame(21249)]

ref_date = datetime.datetime(2022, 12, 20)
sec_date = datetime.datetime(2022, 12, 8)

select_ifg_pair_from_stack(ref_date, sec_date, df_stack_subset, None)

{'reference': ['S1A_IW_SLC__1SDV_20221220T015822_20221220T015852_046409_058F3D_D201',
  'S1A_IW_SLC__1SDV_20221220T015850_20221220T015918_046409_058F3D_2768',
  'S1A_IW_SLC__1SDV_20221220T015916_20221220T015943_046409_058F3D_8C7E'],
 'secondary': ['S1A_IW_SLC__1SDV_20221208T015823_20221208T015853_046234_058941_FEE4',
  'S1A_IW_SLC__1SDV_20221208T015851_20221208T015919_046234_058941_A19D',
  'S1A_IW_SLC__1SDV_20221208T015917_20221208T015944_046234_058941_52CD'],
 'reference_date': datetime.datetime(2022, 12, 20, 0, 0),
 'secondary_date': datetime.datetime(2022, 12, 8, 0, 0),
 'frame_id': None,
 'geometry': None}

In [28]:
S1Frame(21248)

S1Frame(frame_id=21248, track_numbers=[137], frame_geometry=<POLYGON Z ((-120.849 33.739 0, -120.891 33.919 0, -120.926 34.086 0, -121.0...>, use_natural_earth_land_mask=True, coverage_geometry=<POLYGON Z ((-118.285 34.757 0, -118.066 33.652 0, -118.411 33.741 0, -118.5...>)